In [1]:
import torch
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration, T5Model,
    BertForQuestionAnswering, BertTokenizer,
    pipeline
)

In [2]:
class NLPTasks:
    def __init__(self):
        # Initialize models and tokenizers
        self.llama_pipe = pipeline("text-generation",
            model="meta-llama/Llama-3.2-3B",
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        print("\n\n\n 6 \n\n\n")

        self.t5_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large") #783M
        self.t5_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")
        print("\n\n\n 1 \n\n\n")

        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn") #406M
        print("\n\n\n 3 \n\n\n")

        self.bert_tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad") #335M
        self.bert_model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
        print("\n\n\n 2 \n\n\n")

        # Changed translation model to t5-base
        self.t5_translator = T5ForConditionalGeneration.from_pretrained("google-t5/t5-base", device_map="cpu") #223M
        self.t5_translator_tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-base")
        print("\n\n\n 4 \n\n\n")

        self.t5_embeddings = T5Model.from_pretrained("t5-small") #60.5M
        self.t5_embeddings_tokenizer = T5Tokenizer.from_pretrained("t5-small")
        print("\n\n\n 5 \n\n\n")



    def text_to_text_generation(self, input_text, max_length=75):
        input_ids = self.t5_tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
        output_ids = self.t5_model.generate(input_ids, max_length=max_length)
        return self.t5_tokenizer.decode(output_ids[0])

    def question_answering(self, question, context):
        inputs = self.bert_tokenizer.encode_plus(question, context, return_tensors="pt")
        input_ids = inputs["input_ids"].tolist()[0]

        outputs = self.bert_model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits)

        answer_tokens = input_ids[start_idx:end_idx+1]
        return self.bert_tokenizer.decode(answer_tokens)

    def text_summarization(self, article, max_length=130, min_length=30):
        return self.summarizer(article, max_length=max_length, min_length=min_length, do_sample=False)

    def text_translation(self, text, max_length=150, min_length=30):
        inputs = self.t5_translator_tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        output_ids = self.t5_translator.generate(
            inputs["input_ids"],
            max_length=max_length,
            min_length=min_length,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        return self.t5_translator_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    def get_embeddings(self, text):
        input_ids = self.t5_embeddings_tokenizer(text, return_tensors="pt").input_ids
        with torch.no_grad():
            outputs = self.t5_embeddings.encoder(input_ids=input_ids)
            embeddings = outputs.last_hidden_state
        return embeddings

    def text_generate(self, prompt, max_length=100):
        return self.llama_pipe(prompt, max_length=max_length)[0]['generated_text']


In [3]:
nlp = NLPTasks()

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]




 5 





In [5]:

# Text-to-text generation
input_text = "rewrite in simple english: When preparing for exams, it's essential to create a structured study schedule. In Delhi, TopMarks Tutoring offers sessions that teach time management and exam techniques, ensuring students balance study time with breaks."
print("Text-to-text generation:", nlp.text_to_text_generation(input_text))

print("\n\n\n")
# Question answering
question = "What is the capital of France?"
context = "France is a country in Europe. The capital of France is Paris."
print("Question answering:", nlp.question_answering(question, context))
print("\n\n\n")

# Summarization
article = "New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York. A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband. Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared 'I do' five more times, sometimes only within two weeks of each other. In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her 'first and only' marriage. Barrientos, now 39, is facing two criminal counts of 'offering a false instrument for filing in the first degree,' referring to her false statements on the 2010 marriage license application, according to court documents."
print("Summarization:", nlp.text_summarization(article))
print("\n\n\n")

print("text gen: ", nlp.text_generate("in a valley full of unicorns"))
print("\n\n\n")

# Translation
text_to_translate = "translate to french: Artificial Intelligence (AI) is a field of study that focuses on the creation of intelligent machines that work and react like humans. Some of the activities computers with AI are designed for include speech recognition, learning, planning, and problem-solving. AI has become a crucial part of the technology industry."
print("Translation:", nlp.text_translation(text_to_translate))



Text-to-text generation: <pad> When preparing for exams, it's essential to create a structured study schedule. In Delhi, TopMarks Tutoring offers sessions that teach time management and exam techniques, ensuring students balance study time with breaks.</s>




Question answering: paris






Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Summarization: [{'summary_text': "Liana Barrientos, now 39, got married five times in less than a decade. She declared 'I do' five times, sometimes only within two weeks of each other."}]




text gen:  in a valley full of unicorns, a unicorn and a donkey meet. the donkey says, "you are so ugly, i hope i never see you again." the unicorn says, "oh, donkey, you are so ugly, i hope i never see you again." the donkey says, "oh, unicorn, you are so ugly, i hope i never see you again." the unicorn says, "oh, donkey, you are so ugly, i hope i




Translation: L'intelligence artificielle (AI) est un domaine d'études qui met l'accent sur la création de machines intelligentes qui fonctionnent et réagissent comme les humains. Parmi les activités pour lesquelles les ordinateurs avec AI sont conçus, mentionnons la reconnaissance de la parole, l'apprentissage, la planification et la résolution de problèmes. L'AI est devenue une partie cruciale de l'industrie technologique.
